<a href="https://colab.research.google.com/github/samuelfernandof/agentics_workflow_ai/blob/main/agentics_workflow_teste.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Inicializar SQLite para persistência do estado do grafo na memória.
from langchain_core.savers import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

# Definir o estado do agente como uma estrutura de dados com TypedDict
from typing import List, TypedDict

class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    sources: List[str]
    revision_number: int
    max_revisions: int

In [ ]:
# Definir os prompts para cada estágio
PLAN_PROMPT = """Você é um especialista em criação de planos de negócios. Redija um esboço de alto nível sobre o tema fornecido pelo usuário. Incluir anotações e instruções relevantes."""

WRITER_PROMPT = """Você é um assistente de relatórios profissionais. Gere o melhor relatório possível com base na solicitação do usuário e no esboço fornecido. Considere as críticas quando houver e utilize todas as informações abaixo:

------
{content}

Fontes:
{sources}"""

REFLECTION_PROMPT = """Você é um consultor sênior encarregado de revisar o relatório de negócios. Forneça recomendações detalhadas sobre o conteúdo, estilo e profundidade do texto, apontando melhorias claras."""

RESEARCH_PLAN_PROMPT = """Você é um pesquisador responsável por obter dados para um relatório de negócios. Crie três consultas de pesquisa para coletar informações relevantes."""

RESEARCH_REVISOR_PROMPT = """Você é um pesquisador que revisa as informações sugeridas. Crie até três consultas para melhorar as revisões solicitadas."""

DATA_ANALYSIS_PROMPT = """Você é um assistente encarregado de extrair informações de documentos relacionados à tarefa fornecida. Concentre-se nos dados mais relevantes para a análise."""

In [ ]:
# Inicializar TavilyClient para busca avançada de informações
from tavily import TavilyClient

tavily = TavilyClient(api_key="sua-chave")

In [ ]:
import os
from langchain_core.models import SystemMessage, HumanMessage

# Função para ler documentos de uma pasta específica
def read_documents_from_folder(folder_path):
    documents = []
    sources = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):  # Extensão de arquivo, pode ser ajustada para outros formatos
            with open(os.path.join(folder_path, filename), 'r') as file:
                documents.append(file.read())
                sources.append(filename)
    return documents, sources

# Nó de planejamento
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

# Nó de pesquisa para coletar informações
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state.get('content', [])
    sources = state.get('sources', [])

    # Utilizar Tavily para pesquisar
    for query in queries.queries:
        response = tavily.search(query=query, max_results=3)  # Melhoria: 3 resultados por query
        for r in response['results']:
            content.append(r['content'])
            sources.append(r.get('source', 'Fonte desconhecida'))

    return {"content": content, "sources": sources}

# Nó para análise de documentos internos
def document_analysis_node(state: AgentState):
    folder_path = "company_docs"  # Pasta onde os documentos da empresa estão armazenados
    documents, sources = read_documents_from_folder(folder_path)

    content = state.get('content', [])
    for document in documents:
        messages = [
            SystemMessage(content=DOCUMENT_ANALYSIS_PROMPT),
            HumanMessage(content=document)
        ]
        response = model.invoke(messages)
        content.append(response.content)

    # Atualizar as fontes
    all_sources = state.get('sources', [])
    all_sources.extend(sources)

    return {"content": content, "sources": all_sources}

# Nó de geração de rascunho
def generation_node(state: AgentState):
    content = "\n\n".join(state.get('content', []))
    sources = "\n".join(state.get('sources', []))
    user_message = HumanMessage(content=f"{state['task']}\n\nAqui está o plano:\n\n{state['plan']}")

    messages = [
        SystemMessage(content=WRITER_PROMPT.format(content=content, sources=sources)),
        user_message
    ]
    response = model.invoke(messages)

    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1
    }

# Nó de reflexão e crítica
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

# Nó de pesquisa com base nas críticas
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state.get('content', [])
    sources = state.get('sources', [])

    # Pesquisar as críticas
    for query in queries.queries:
        response = tavily.search(query=query, max_results=3)
        for r in response['results']:
            content.append(r['content'])
            sources.append(r.get('source', 'Fonte desconhecida'))

    return {"content": content, "sources": sources}

# Verificação para continuar ou parar
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return "END"
    return "reflect"

In [ ]:
from langchain_core.graph import StateGraph

# Criar o gráfico de estados
builder = StateGraph(AgentState)

builder.add_node("plan", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("document_analysis", document_analysis_node)
builder.add_node("research_critique", research_critique_node)

builder.set_entry_point("plan")
builder.add_conditional_edges(
    "generate",
    should_continue,
    {"END": "END", "reflect": "reflect"}
)

builder.add_edge("plan", "research_plan")
builder.add_edge("research_plan", "document_analysis")
builder.add_edge("document_analysis", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

graph = builder.compile(checkpointer=memory)

#Visualizar grafo
from IPython.display import Image

# Exibir o grafo
Image(graph.get_graph().draw_png())

In [ ]:
# Definir o estado inicial e iniciar o gráfico
thread = {"configurable": {"thread_id": "1"}}

# Definir a tarefa inicial do agente
initial_state = {
    'task': "A Take-it-easy Inc. está enfrentando uma queda nas vendas de sua principal linha de produtos, que está no mercado há 10 anos. Eles suspeitam de aumento da concorrência e estratégias de marketing desatualizadas. O resultado desejado é revitalizar as vendas e recuperar a participação de mercado nos próximos 6 meses. Eles estão abertos a explorar novos canais de marketing. Gere um relatório de exemplo de 2 páginas em formato markdown, com um teaser de possíveis soluções e como a LN Consulting pode ajudar, listando estudos de caso relevantes, links de pesquisa, e nomes de documentos.",
    "max_revisions": 2,  # Número máximo de revisões
    "revision_number": 1  # Começar da primeira revisão
}

# Executar o gráfico com base no estado inicial
for s in graph.stream(initial_state, thread):
    print(s)

from langtrace_python_sdk import langtrace

# Inicializar rastreamento com LangTrace
langtrace.init(api_key='sua_chave')

# Adicione rastreamento ao gráfico
for s in graph.stream(initial_state, thread):
    langtrace.record_state(s)
    print(s)